In [126]:
import pandas as pd

In [127]:
# Load the data
df_2011 = pd.read_csv('data/2011.csv')
df_2013 = pd.read_csv('data/2013.csv')
df_2014 = pd.read_csv('data/2014.csv')
df_2016 = pd.read_csv('data/2016.csv')
df_2017 = pd.read_csv('data/2017.csv')
df_2018 = pd.read_csv('data/2018.csv')
df_2019 = pd.read_csv('data/2019.csv')
df_2020 = pd.read_csv('data/2020.csv')
df_2022 = pd.read_csv('data/2022.csv')

In [128]:
# Concatenate the data with only the columns that are present in the df_2011
df = pd.concat([
    df_2011,
    df_2013,
    df_2014,
    df_2016,
    df_2017,
    df_2018,
    df_2019,
    df_2020,
    df_2022
], axis=0, ignore_index=True)

In [129]:
df.head()

,dia_ocu,mes_ocu,dia_sem_ocu,hora_ocu,depto_ocu,zona_ocu,sexo_pil,edad_pil,mayor_menor,estado_pil,tipo_vehi,marca_vehi,causa_acc,year
0,27,Junio,Lunes,17,Guatemala,10,Hombre,42,Mayor,Normal,Automovil,Audi,Colisión,2011
1,18,Septiembre,Domingo,0,Guatemala,7,Hombre,25,Mayor,Ebrio,Automovil,Audi,Empotro,2011
2,5,Octubre,Miercoles,19,Quetzaltenango,Ignorado,Hombre,26,Mayor,Ebrio,Automovil,Audi,Colisión,2011
3,15,Febrero,Martes,12,Izabal,Ignorado,Hombre,59,Mayor,Ignorado,Camioneta,Land rover,Colisión,2011
4,17,Junio,Viernes,18,Guatemala,11,Mujer,25,Mayor,Normal,Camioneta,Land rover,Colisión,2011
